#### Import Dependencies 

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from tqdm import tqdm
import copy
import pickle
from itertools import product
import pickle
from pytorchtools import EarlyStopping

In [2]:
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True 

#### Load Dataset
- Data must be in the following shape: 
    - X.shape = (number of trials for all subjects, timestamp, number of channels)
    - Y.shape = (number of trials for all subjects, 2) (shape of 2 for an array([subject id, label])

In [3]:
X, Y = pickle.load(open('CrossSubjects.pkl', "rb"))

In [4]:
X.shape, Y.shape

((2328, 1750, 22), (2328, 2))

#### Define Variables

In [5]:
subject = 0 # subject to leave out from training and then test on 
total_subjects = 9 #total number of subjects in dataset
nchannels = 22 # number of EEG channels
batch_size = 12 # batch size for training 
batch_size1 = 1 # batch size for testing 
num_epoch = 1000 # number of epochs for training 
lr = 1e-4 #learning rate 
patience = 50 # patience for early stopping
timestamp = 1750 #timestamp recorded in trials
nclasses = 4 #number of label classes 

#### Data Preprocessing

In [6]:
#Specify which subject you want to leave out 
def subject_out(X,Y,subject=0):
    XS = []
    YS = []
    XL = []
    YL = []
    for i in range(len(X)):
        if Y[i][0] == subject:
            XL.append(X[i])
            YL.append(Y[i])
        else:
            XS.append(X[i])
            YS.append(Y[i])
    XS = np.array(XS)
    YS = np.array(YS)
    XL = np.array(XL)
    YL = np.array(YL)
    return XS, YS, XL, YL

In [7]:
#Normalize the data per the number of EEG channels 
def prepsubdata(X,Y,nchannels):
    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=0)
    
    scaler = StandardScaler()
    scaler.fit(X_train.reshape([-1,nchannels]))
    X1_train = scaler.transform(X_train.reshape([-1,nchannels])).reshape(X_train.shape)
    X1_test = scaler.transform(X_test.reshape([-1,nchannels])).reshape(X_test.shape)
    
    ########
    
    X1_train = torch.FloatTensor(X1_train).cuda()
    Y1_train = torch.FloatTensor(y_train).cuda()
    X1_test = torch.FloatTensor(X1_test).cuda()
    Y1_test = torch.FloatTensor(y_test).cuda()

    
    return X1_train, X1_test, Y1_train, Y1_test, scaler

def preploaders(X,Y,batch_size):
    X_train, X_test, Y_train, Y_test, scaler = prepsubdata(X,Y,nchannels)
    train_data =[]
    for i in range(len(X_train)):
        train_data.append([X_train[i], Y_train[i]])
    test_data =[]
    for i in range(len(X_test)):
        test_data.append([X_test[i], Y_test[i]])
    
    trainloader = DataLoader(train_data, batch_size) 
    testloader = DataLoader(test_data, batch_size)    
    
    return trainloader, testloader, scaler


In [8]:
#Load the data for the test subject 
def prepdataloader(X,Y,scaler,nchannels,batch_size1):
    
    #Normalize
    XT = scaler.transform(X.reshape([-1,nchannels])).reshape(X.shape)
    
    XT = torch.FloatTensor(XT).cuda()
    Y = torch.FloatTensor(Y).cuda()
    

    
    #Prep Loader
    data =[]
    for i in range(len(XT)):
        data.append([XT[i], Y[i]])
    
    xloader = DataLoader(data,batch_size1)
    
    return xloader
 


#### Define Network

In [9]:
class EEGNet1(nn.Module):
    def __init__(self):
        super(EEGNet1, self).__init__()
        self.T = timestamp
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, nchannels), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16, False)
        self.dropout1 = nn.Dropout(p=0.25)
        self.elu = nn.ELU()
      
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.batchnorm2 = nn.BatchNorm2d(4, False)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4, False)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer 
        self.fc1 = nn.Linear(4*2*109, nclasses)

        

    def forward(self, x):
        # Layer 1
        x = self.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = self.dropout1(x)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = self.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = self.dropout1(x)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = self.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = self.dropout1(x)
        x = self.pooling3(x)
   
        
        # FC Layer
        x = x.reshape([x.shape[0],x.shape[1]*x.shape[2]*x.shape[3]]) #flatten
        x = self.fc1(x) #removed sigmoid 
        return x 

#### Train & Test Model
- Run the following for cross-subject training/testing
- Can try out different learning rates

In [10]:
for  lr, subject in product([lr], range(total_subjects)): #iteratively selects subject to leave out
    torch.manual_seed(0)
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    np.random.seed(0)
    random.seed(0)

    XS, YS, XL, YL = subject_out(X,Y,subject)
    
    model = EEGNet1().cuda()
    criterion = nn.CrossEntropyLoss()


    optimizer = optim.Adam(model.parameters() , lr=lr)

    
    trainingSUbjects = list(range(nchannels))
    trainingSUbjects.remove(subject)
 


    trainLoss = [] 
    valLoss = []
    train_acc = []
    val_acc = []

    early_stopping = EarlyStopping(patience, verbose=True)

 

    trainloader, testloader, scaler = preploaders(XS, YS, batch_size)
    


    for epoch in tqdm(range(num_epoch)):  
        training_loss = 0
        validation_loss = 0
        total_train = 0
        correct_train = 0


        model.train()

        for i, data in enumerate(trainloader,0):
            inputs, labels = data

        
            optimizer.zero_grad()
            inputNew = inputs[:,np.newaxis,:,:]
   
    
            outputs = model(inputNew)
            loss = criterion(outputs, labels[:,1].long())
            loss.backward()
            optimizer.step()

            training_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)

            correct_train += (predicted == labels[:,1].long()).sum().item()



        correct = 0
        total = 0

        model.eval() 

        with torch.no_grad():
            for data in testloader:
                inputs, labels = data
                inputNew = inputs[:,np.newaxis,:,:]


                outputs = model(inputNew)

                loss = criterion(outputs, labels[:,1].long()) 
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels[:,1].long()).sum().item()
                validation_loss += loss.item()



        acc_train = 100 * correct_train // total_train
        acc_val = 100 * correct // total
        training_loss = training_loss/(len(trainloader))
        validation_loss = validation_loss/(len(testloader))
        trainLoss.append(training_loss)
        valLoss.append(validation_loss)
        train_acc.append(acc_train)
        val_acc.append(acc_val)
        print(f"Training Loss: {training_loss}")
        print(f"Validation Loss: {validation_loss}")
        print(f'Training Accuracy of the network: {acc_train} %')
        print(f'Validation Accuracy of the network: {acc_val} %')


        early_stopping(validation_loss,model)
        if early_stopping.bestFound:
            iterationOfBestModel = epoch
            
        if early_stopping.early_stop:
            print("Early Stopping")
            break
    model = early_stopping.bestModel





    XL_train, XL_test, YL_train, YL_test = train_test_split(XL,YL, test_size=0.95, random_state=0)
    loader = prepdataloader(XL_train,YL_train,scaler,nchannels,batch_size1)
    subjectloader = prepdataloader(XL_test,YL_test,scaler,nchannels,batch_size1=XL_test.shape[0])

        
    

    model.eval()
    testLoss = []
    test_acc = []
        

     



    correct_test = 0
    total_test = 0
    testing_loss = 0

    with torch.no_grad():
        for data in subjectloader:
            inputs, labels = data
            inputNew = inputs[:,np.newaxis,:,:]
        
            outputs = model(inputNew)

            loss = criterion(outputs, labels[:,1].long()) 
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels[:,1].long()).sum().item()
            testing_loss += loss.item()

    acc_test = 100 * correct_test // total_test
    testing_loss = testing_loss/(len(subjectloader))
    testLoss.append(testing_loss)
    test_acc.append(acc_test)
    print(f"Testing Loss: {testing_loss}")
    print(f'Testing Accuracy of the network: {acc_test} %')  

  0%|▏                                                                                                                                                                 | 1/1000 [00:04<1:13:57,  4.44s/it]

Training Loss: 1.5586490274345788
Validation Loss: 1.4326368195669992
Training Accuracy of the network: 24 %
Validation Accuracy of the network: 23 %
Validation loss decreased (inf --> 1.432637).  Saving model ...


  0%|▎                                                                                                                                                                   | 2/1000 [00:06<55:19,  3.33s/it]

Training Loss: 1.5078786559348558
Validation Loss: 1.4208858592169626
Training Accuracy of the network: 26 %
Validation Accuracy of the network: 24 %
Validation loss decreased (1.432637 --> 1.420886).  Saving model ...


  0%|▍                                                                                                                                                                   | 3/1000 [00:09<49:31,  2.98s/it]

...

Training Loss: 0.4117583424784243
Validation Loss: 1.1548416784831457
Training Accuracy of the network: 83 %
Validation Accuracy of the network: 52 %
EarlyStopping counter: 48 out of 50


 13%|█████████████████████▍                                                                                                                                            | 132/1000 [05:38<36:50,  2.55s/it]

Training Loss: 0.41789051563079865
Validation Loss: 1.1573468582970756
Training Accuracy of the network: 84 %
Validation Accuracy of the network: 51 %
EarlyStopping counter: 49 out of 50


 13%|█████████████████████▍                                                                                                                                            | 132/1000 [05:40<37:20,  2.58s/it]

Training Loss: 0.40656701815979823
Validation Loss: 1.177463928290776
Training Accuracy of the network: 83 %
Validation Accuracy of the network: 52 %
EarlyStopping counter: 50 out of 50
Early Stopping
Testing Loss: 1.5091813802719116
Testing Accuracy of the network: 47 %
